# Colab Start Section

Step 1 is to upload a copy of your private key. I am assuming your private key is named id_ed25519. If that isn't the case, you'll have to make manual changes.

In [1]:
# Change these of course to what is your email and your GitHub username
EMAIL = "michael.murray.iv@gmail.com"
USERNAME = "michaelmurrayiv"
ORG = "anderson-fall-2024"

In [2]:
from google.colab import files
uploaded = files.upload()

Saving id_ed25519 to id_ed25519


In [3]:
%%bash -s "$EMAIL" "$USERNAME"

rm -rf /root/.ssh/*
mkdir /root/.ssh

mv id_ed25519 /root/.ssh/

chmod 600 /root/.ssh/id_ed25519

git config --global user.email "$1"
git config --global user.name "$2"

In [8]:
%%bash -s "$USERNAME" "$ORG"

eval "$(ssh-agent -s)"

ssh-add ~/.ssh/id_ed25519

ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

git clone git@github.com:$2/csc-487-$1.git

Agent pid 726


Identity added: /root/.ssh/id_ed25519 (pauleanderson@gmail.com)
# github.com:22 SSH-2.0-babeld-cd94d92b7
Cloning into 'csc-487-instructor'...


In [2]:
%load_ext autoreload
%autoreload 2


from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

import sys
sys.path.insert(0,f"csc-487-{USERNAME}")
sys.path.insert(0,"..")

import py487

In [3]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Exercises
========================

In [4]:
import pandas as pd
data = pd.DataFrame([[0,0,1],[0,1,1.5],[1,0,1.75],[1,1,2.25]],columns=['X1','X2','t'])
data

,X1,X2,t
0,0,0,1.00
1,0,1,1.50
2,1,0,1.75
3,1,1,2.25


In [5]:
X = data.iloc[:,:2]
X

,X1,X2
0,0,0
1,0,1
2,1,0
3,1,1


Because this is a learning exercise, here are the answers we are going to hunt for using gradient descent. In other words, your last theta should be w and b given below.

In [6]:
import numpy as np
w = np.array([0.75,0.5])
b = 1
y = np.dot(X,w)+b
y

array([1.  , 1.5 , 1.75, 2.25])

Our y functions are:

$y_1 = 0*w_1+0*w_2+b$

$y_2 = 0*w_1+1*w_2+b$

$y_3 = 1*w_1+0*w_2+b$

$y_4 = 1*w_1+1*w_2+b$

Our Loss functions are:

$L_1 = (b - 1)^2$

$L_2 = (1*w_2+b - 1.5)^2$

$L_3 = (1*w_1+b - 1.75)^2$

$L_4 = (1*w_1+1*w_2+b - 2.25)^2$

Our derivatives are:

$\frac{\delta L_1}{\delta w1} = 0$

$\frac{\delta L_2}{\delta w1} = 0$

$\frac{\delta L_3}{\delta w1} = 2(w_1+b-1.75)$

$\frac{\delta L_4}{\delta w1} = 2(w_1+w_2+b-2.25)$

$\frac{\delta L_1}{\delta w2} = 0$

$\frac{\delta L_2}{\delta w2} = 2(w_2+b-1.5)$

$\frac{\delta L_3}{\delta w2} = 0$

$\frac{\delta L_4}{\delta w2} = 2(w_1+w_2+b-2.25)$

$\frac{\delta L_1}{\delta b} = 2(b-1)$

$\frac{\delta L_2}{\delta b} = 2(w_2+b-1.5)$

$\frac{\delta L_3}{\delta b} = 2(w_1+b-1.75)$

$\frac{\delta L_4}{\delta b} = 2(w_1+w_2+b-2.25)$

## Gradient descent

Our first take on gradient descent will assume you can symbolically find the gradient of a function. So you can use the derivatives I supplied above:

Gradient descent says:

$w_1 = w_1 - \alpha \frac{1}{4} \left(\sum_{i=1}^4\frac{dF_i}{dw_1}\right)$

$w_2 = w_2 - \alpha \frac{1}{4} \left(\sum_{i=1}^4\frac{dF_i}{dw_2}\right)$

$b = b - \alpha \frac{1}{4} \left(\sum_{i=1}^4\frac{dF_i}{db}\right)$

We will set $\alpha=0.1$.

We will now define the derivatives programmatically:

In [7]:
import random
gradients_w1 = [lambda w1,w2,b: 0, lambda w1,w2,b: 0, lambda w1,w2,b: 2*(w1+b-1.75), lambda w1,w2,b: 2*(w1+w2+b-2.25)]
print('This should be all zeros:',[gradients_w1[i](0.75,0.5,1) for i in range(4)])
print('This should be non-zero:',[gradients_w1[i](0.75-random.random(),0.5-random.random(),1-random.random()) for i in range(4)])
print('This should be non-zero:',[gradients_w1[i](0.75+random.random(),0.5+random.random(),1+random.random()) for i in range(4)])

This should be all zeros: [0, 0, 0.0, 0.0]
This should be non-zero: [0, 0, -2.926025442982376, -4.442400020765511]
This should be non-zero: [0, 0, 2.5355303743552327, 3.432137197518486]


In [8]:
import random
gradients_w2 = [lambda w1,w2,b: 0, lambda w1,w2,b: 2*(w2+b-1.5), lambda w1,w2,b: 0, lambda w1,w2,b: 2*(w1+w2+b-2.25)]
print('This should be all zeros:',[gradients_w2[i](0.75,0.5,1) for i in range(4)])
print('This should be non-zero:',[gradients_w2[i](0.75-random.random(),0.5-random.random(),1-random.random()) for i in range(4)])
print('This should be non-zero:',[gradients_w2[i](0.75+random.random(),0.5+random.random(),1+random.random()) for i in range(4)])

This should be all zeros: [0, 0.0, 0, 0.0]
This should be non-zero: [0, -3.7466634782262265, 0, -4.3704950511396214]
This should be non-zero: [0, 2.595405320758542, 0, 2.13950036374942]


In [9]:
import random
gradients_b = [lambda w1,w2,b: 2*(b-1), lambda w1,w2,b: 2*(w2+b-1.5), lambda w1,w2,b: 2*(w2+b-1.5), lambda w1,w2,b: 2*(w1+w2+b-2.25)]
print('This should be all zeros:',[gradients_b[i](0.75,0.5,1) for i in range(4)])
print('This should be non-zero:',[gradients_b[i](0.75-random.random(),0.5-random.random(),1-random.random()) for i in range(4)])
print('This should be non-zero:',[gradients_b[i](0.75+random.random(),0.5+random.random(),1+random.random()) for i in range(4)])

This should be all zeros: [0, 0.0, 0.0, 0.0]
This should be non-zero: [-0.5263713351804915, -1.6536549887850418, -0.7803933143618815, -2.89325814097763]
This should be non-zero: [0.9809002420383983, 1.9924939485126858, 2.2339298112153383, 3.2178543607676486]


In [10]:
thetas = py487.gradient_descent.minimize_gradient_descent([gradients_w1,gradients_w2,gradients_b],0.1,[0.5,-0.2,2.5])
pd.Series(thetas)
# please note that I only add the pd.Series, so the output is nicely formatted

0                                       [0.5, -0.2, 2.5]
1                 [0.41000000000000003, -0.2675, 2.3175]
2                          [0.350625, -0.3055, 2.186125]
3                    [0.312225, -0.32359375, 2.08969375]
4      [0.28821281249999997, -0.328315, 2.01718281249...
                             ...                        
496    [0.7499998348770656, 0.49999983487304656, 1.00...
497    [0.7499998400594908, 0.4999998400556728, 1.000...
498    [0.7499998450792664, 0.4999998450756393, 1.000...
499    [0.749999849941497, 0.4999998499380513, 1.0000...
500    [0.7499998546511271, 0.4999998546478536, 1.000...
Length: 501, dtype: object

In [11]:
print('This should be all zeros, but is it...:',[gradients_w1[i](0.75,0.5,1) for i in range(4)])
print('This should be all zeros, but is it...:',[gradients_w2[i](0.75,0.5,1) for i in range(4)])
print('This should be all zeros, but is it...:',[gradients_b[i](0.75,0.5,1) for i in range(4)])

This should be all zeros, but is it...: [0, 0, 0.0, 0.0]
This should be all zeros, but is it...: [0, 0.0, 0, 0.0]
This should be all zeros, but is it...: [0, 0.0, 0.0, 0.0]


In [12]:
import numpy as np
w_predicted = np.array(thetas[-1][:2])
b_predicted = thetas[-1][-1]
y = np.dot(X,w_predicted)+b_predicted
y

array([1.00000017, 1.50000003, 1.75000003, 2.24999988])

Now what if you can't or don't want to find the derivatives symbolically? 
You can always estimate the gradient analytically using the difference quotient:

$[L(\theta+h)-L(\theta)]/h$,

where h is a scalar parameter. Let's give it a shot with our functions. As a reminder, they are:

$L_1 = (b - 1)^2$

$L_2 = (1*w_2+b - 1.5)^2$

$L_3 = (1*w_1+b - 1.75)^2$

$L_4 = (1*w_1+1*w_2+b - 2.25)^2$

In [13]:
F1_func = lambda w1,w2,b: (b-1)**2
F1_func(w[0],w[1],b)

0

In [14]:
F2_func = lambda w1,w2,b: (w2+b-1.5)**2
F2_func(w[0],w[1],b)

np.float64(0.0)

In [15]:
F3_func = lambda w1,w2,b: (w1+b-1.75)**2
F3_func(w[0],w[1],b)

np.float64(0.0)

In [16]:
F4_func = lambda w1,w2,b: (w1+w2+b-2.25)**2
F4_func(w[0],w[1],b)

np.float64(0.0)

In [17]:
R_func = lambda w1,w2,b: 1/4*(F1_func(w1,w2,b)+F2_func(w1,w2,b)+F3_func(w1,w2,b)+F4_func(w1,w2,b))
R_func(w[0],w[1],b)

np.float64(0.0)

In [18]:
thetas = py487.gradient_descent.minimize_gradient_descent_analytically(R_func,0.1,[0.5,-0.2,2.5],0.01)
pd.Series(thetas)

iteration,gradient,theta,previous theta
0,2.0599999999999508,[0.4095000000000022, -0.2680000000000014, 2.294000000000005],[0.5, -0.2, 2.5]
10,0.25526254996852404,[0.3175040100399055, -0.18845603148450446, 1.6736448970651698],[0.30817593425087564, -0.2112556884063983, 1.6991711520620223]
20,0.15876377392169405,[0.4157861223593313, 0.012533390529677085, 1.4826129357364803],[0.4065519063741218, -0.004766758709724678, 1.4984893131286499]
30,0.11483596968678081,[0.4971432023882691, 0.15538513080266153, 1.3493491614749202],[0.48987894828695583, 0.143291504512632, 1.3608327584435984]
40,0.08346814430347793,[0.5603002897445661, 0.25536134281298334, 1.2525933382540761],[0.5547091546944632, 0.24687868424016565, 1.260940152684424]
50,0.060677341937009216,[0.6085746621819432, 0.3256806852511477, 1.182258788550177],[0.604324767661888, 0.31969952878736635, 1.188326522743878]
60,0.044109710356573725,[0.6450812922665422, 0.3753864531996199, 1.1311287701943042],[0.6418808187203902, 0.3711494091762614, 

0                                       [0.5, -0.2, 2.5]
1      [0.4095000000000022, -0.2680000000000014, 2.29...
2      [0.3520500000000022, -0.3040750000000035, 2.14...
3      [0.3170437500000002, -0.3187750000000043, 2.03...
4      [0.29725387500000044, -0.31927393750000405, 1....
                             ...                        
496    [0.7499998915095999, 0.49999989150558083, 0.99...
497    [0.749999894914635, 0.4999998949108169, 0.9950...
498    [0.7499998982128018, 0.4999998982091746, 0.995...
499    [0.7499999014074543, 0.49999990140400846, 0.99...
500    [0.7499999045018414, 0.4999999044985678, 0.995...
Length: 501, dtype: object

In [19]:
R_func(*thetas[-1]) # Shouldn't be too bad :)

2.4999822240924255e-05

In [20]:
# Good job!
# Woohoo!

# Test your code

In [21]:
%%bash -s "$USERNAME"

cd ../tests #csc-487-$1/tests
pytest test_Assignment4.py

============================= test session starts ==============================
platform darwin -- Python 3.9.6, pytest-8.3.3, pluggy-1.5.0
rootdir: /Users/pander14/GitHub/csc-487-instructor/tests
plugins: anyio-4.6.0
collected 2 items

test_Assignment4.py ..                                                   [100%]

============================== 2 passed in 0.85s ===============================


# Colab End Section - Submit your code

In [35]:
%%bash -s "$USERNAME"

eval "$(ssh-agent -s)"

ssh-add ~/.ssh/id_ed25519

ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

cd csc-487-$1
git add .
git commit -m update
git push
#./command_line_sync.sh # if you want to sync

Agent pid 5184
[main 5849bef] update
 2 files changed, 42 insertions(+)
 create mode 100644 tests/answers_Assignment3.joblib
 create mode 100644 tests/test_Assignment3.py


Identity added: /root/.ssh/id_ed25519 (pauleanderson@gmail.com)
# github.com:22 SSH-2.0-babeld-cd94d92b7
remote: This repository moved. Please use the new location:        
remote:   git@github.com:Anderson-Lab/csc-487-instructor.git        
To github.com:anderson-lab/csc-487-instructor.git
   b93bd8a..5849bef  main -> main
